In [8]:
!pip install scikit-learn
!pip install seaborn
!pip install matplotlib

In [9]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import re

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [10]:
# Punto --> Exploración de los datos! ✅

In [11]:
df = pd.read_csv('df_superlimpiooo.csv', index_col= 0)
df.head()

,age,attrition,businesstravel,dailyrate,distancefromhome,education,educationfield,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,sameasmonthlyincome,datebirth,remotework
employeenumber,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"162,0",51,No,NaN,684.0,6,3,NaN,1,Male,51,3,5,research director,3,NaN,195370.0,6462,7,No,13,3.0,3,0,NaN,5,3.0,20,15,15,19537.0,1972,Yes
"259,0",52,No,NaN,699.0,1,4,life sciences,3,Male,65,2,5,manager,3,NaN,199990.0,5678,0,NaN,14,3.0,1,1,34.0,5,3.0,33,11,9,19999.0,1971,Yes
"319,0",42,No,travel rarely,532.0,4,2,technical degree,3,Male,58,3,5,manager,4,married,192320.0,4933,1,No,11,3.0,4,0,22.0,3,NaN,22,11,15,19232.0,1981,Yes
NaN,47,No,travel rarely,359.0,2,4,medical,1,Female,82,3,4,research director,3,married,171690.0,26703,3,NaN,19,3.0,2,2,NaN,2,NaN,20,5,6,17169.0,1976,No
NaN,46,No,NaN,1319.0,3,3,technical degree,1,Female,45,4,4,sales executive,1,divorced,NaN,7739,2,No,12,3.0,4,1,NaN,5,3.0,19,2,8,NaN,1977,No


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1614 entries, 162,0 to nan
Data columns (total 32 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   age                       1614 non-null   int64  
 1   attrition                 1614 non-null   object 
 2   businesstravel            842 non-null    object 
 3   dailyrate                 1490 non-null   float64
 4   distancefromhome          1614 non-null   int64  
 5   education                 1614 non-null   int64  
 6   educationfield            869 non-null    object 
 7   environmentsatisfaction   1614 non-null   int64  
 8   gender                    1614 non-null   object 
 9   hourlyrate                1614 non-null   int64  
 10  jobinvolvement            1614 non-null   int64  
 11  joblevel                  1614 non-null   int64  
 12  jobrole                   1614 non-null   object 
 13  jobsatisfaction           1614 non-null   int64  
 14  maritalsta

In [13]:
df.dtypes

age                           int64
attrition                    object
businesstravel               object
dailyrate                   float64
distancefromhome              int64
education                     int64
educationfield               object
environmentsatisfaction       int64
gender                       object
hourlyrate                    int64
jobinvolvement                int64
joblevel                      int64
jobrole                      object
jobsatisfaction               int64
maritalstatus                object
monthlyincome               float64
monthlyrate                   int64
numcompaniesworked            int64
overtime                     object
percentsalaryhike             int64
performancerating           float64
relationshipsatisfaction      int64
stockoptionlevel              int64
totalworkingyears           float64
trainingtimeslastyear         int64
worklifebalance             float64
yearsatcompany                int64
yearssincelastpromotion     

In [15]:
# Obtenemos la lista de columnas categóricas que tienen nulos
nulos_esta_cat = df[df.columns[df.isnull().any()]].select_dtypes(include = "O").columns
print("Las columnas categóricas que tienen nulos son : \n ")
print(nulos_esta_cat)

Las columnas categóricas que tienen nulos son : 
 
Index(['businesstravel', 'educationfield', 'maritalstatus', 'overtime'], dtype='object')


# A) Tratamiento de valores nulos Variables Categoricas


In [16]:
#funcion para calcular el porcentaje de valores nulos en las variables categoricas
def calcular_porcentaje_nulos(df, columnas_categoricas):
    porcentaje_nulos = {}
    for columna in columnas_categoricas:
        # Calcular el número de valores nulos
        nulos = df[columna].isnull().sum()
        # Calcular el porcentaje de valores nulos
        porcentaje = (nulos / len(df)) * 100
        # Guardar el resultado en el diccionario
        porcentaje_nulos[columna] = porcentaje
    return porcentaje_nulos

In [17]:
# Lista de columnas categóricas
columnas_categoricas = ['businesstravel', 'educationfield', 'maritalstatus', 'overtime']

In [18]:
porcentajes_nulos = calcular_porcentaje_nulos(df, columnas_categoricas)

for columna, porcentaje in porcentajes_nulos.items():
    print(f"{columna}: {porcentaje:.2f}%")

businesstravel: 47.83%
educationfield: 46.16%
maritalstatus: 40.33%
overtime: 41.88%


In [ ]:
#Según los porcentajes de las categorías, si hay una categoría dominante reemplazamos por la moda, de lo contrario indicamos que pertenece a la categoría 'unknown'(desconocido)

# educationfield :categoría 'Desconocido' porque tiene casi 47% de nulos.

# maritalstatus categoría 'Desconocido' porque tiene un 40% de nulos. 

#overtime: categoría 'Desconocido' porque tiene un 40% de nulos. 

In [19]:
columnas_categoricas = ['businesstravel','educationfield', 'maritalstatus', 'overtime']

In [20]:
def reemplazar_nulos_por_moda(df, columnas_categoricas):
    for columna in columnas_categoricas:
        # Calcular la moda de la columna
        moda = df[columna].mode()[0]
        # Reemplazar los valores nulos por la moda
        df[columna] = df[columna].fillna(moda)
    return df

In [21]:
df = reemplazar_nulos_por_moda(df, columnas_categoricas)

In [22]:
print("\nDespués de reemplazar los nulos:")
print(df)


Después de reemplazar los nulos:
                age attrition businesstravel  dailyrate  distancefromhome  \
employeenumber                                                              
162,0            51        No  travel rarely      684.0                 6   
259,0            52        No  travel rarely      699.0                 1   
319,0            42        No  travel rarely      532.0                 4   
NaN              47        No  travel rarely      359.0                 2   
NaN              46        No  travel rarely     1319.0                 3   
...             ...       ...            ...        ...               ...   
967,0            36       Yes  travel rarely      530.0                 3   
972,0            45        No     non travel      805.0                 4   
NaN              39        No  travel rarely      903.0                13   
990,0            36        No     non travel     1229.0                 8   
NaN              46        No  travel rare

In [23]:
# Obtenemos la lista de columnas categóricas que tienen nulos
nulos_esta_cat = df[df.columns[df.isnull().any()]].select_dtypes(include = "O").columns
print("Las columnas categóricas que tienen nulos son : \n ")
print(nulos_esta_cat)

Las columnas categóricas que tienen nulos son : 
 
Index([], dtype='object')


In [24]:
df.head(5)

,age,attrition,businesstravel,dailyrate,distancefromhome,education,educationfield,environmentsatisfaction,gender,hourlyrate,jobinvolvement,joblevel,jobrole,jobsatisfaction,maritalstatus,monthlyincome,monthlyrate,numcompaniesworked,overtime,percentsalaryhike,performancerating,relationshipsatisfaction,stockoptionlevel,totalworkingyears,trainingtimeslastyear,worklifebalance,yearsatcompany,yearssincelastpromotion,yearswithcurrmanager,sameasmonthlyincome,datebirth,remotework
employeenumber,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
"162,0",51,No,travel rarely,684.0,6,3,life sciences,1,Male,51,3,5,research director,3,married,195370.0,6462,7,No,13,3.0,3,0,NaN,5,3.0,20,15,15,19537.0,1972,Yes
"259,0",52,No,travel rarely,699.0,1,4,life sciences,3,Male,65,2,5,manager,3,married,199990.0,5678,0,No,14,3.0,1,1,34.0,5,3.0,33,11,9,19999.0,1971,Yes
"319,0",42,No,travel rarely,532.0,4,2,technical degree,3,Male,58,3,5,manager,4,married,192320.0,4933,1,No,11,3.0,4,0,22.0,3,NaN,22,11,15,19232.0,1981,Yes
NaN,47,No,travel rarely,359.0,2,4,medical,1,Female,82,3,4,research director,3,married,171690.0,26703,3,No,19,3.0,2,2,NaN,2,NaN,20,5,6,17169.0,1976,No
NaN,46,No,travel rarely,1319.0,3,3,technical degree,1,Female,45,4,4,sales executive,1,divorced,NaN,7739,2,No,12,3.0,4,1,NaN,5,3.0,19,2,8,NaN,1977,No
